Printeo del nuevo modelo usado

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths

import os

import torch
import torch.utils.data
from opts import opts
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory
import cv2
import sys
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
sys.argv = ['-f', 'fewdetv3', '--exp_id', 'coco_dlav7','--num_workers', '0','--batch_size','2', '--arch','dlav7_34'] #cmd = 'ctdet --exp_id coco_dla --batch_size 4 --lr 1.25e-4  --gpus 0 --num_workers 0'
opt = opts().parse()
print(opt.task)
print(opt.num_workers)
print(opt.batch_size)

Fix size testing.
training chunk_sizes: [2]
The output will be saved to  C:\Users\avata\FewShot-CenterNet\src\lib\..\..\exp\fewdetv3\coco_dlav7
fewdetv3
0
2


In [3]:
torch.manual_seed(opt.seed)
torch.backends.cudnn.benchmark = not opt.not_cuda_benchmark and not opt.test

In [4]:
opt.reg_loss

'l1'

In [5]:
Dataset = get_dataset(opt.dataset, opt.task)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
logger = Logger(opt)
os.environ['CUDA_VISIBLE_DEVICES'] = opt.gpus_str
opt.device = torch.device('cuda' if opt.gpus[0] >= 0 else 'cpu')

heads {'ss': 80, 'hm': 1, 'wh': 2, 'reg': 2}


In [6]:
model = create_model(opt.arch, opt.heads, opt.head_conv)
optimizer = torch.optim.Adam(model.parameters(), opt.lr)
start_epoch = 0
if opt.load_model != '':
    model, optimizer, start_epoch = load_model(model, opt.load_model, optimizer, opt.resume, opt.lr, opt.lr_step)

In [7]:
Trainer = train_factory[opt.task]
trainer = Trainer(opt, model, optimizer)
trainer.set_device(opt.gpus, opt.chunk_sizes, opt.device)

In [8]:
train_loader = torch.utils.data.DataLoader(
      Dataset(opt, 'train'), 
      batch_size=opt.batch_size, 
      shuffle=True,
      num_workers=opt.num_workers,
      pin_memory=True,
      drop_last=True
  )

==> initializing coco 2017 train data.
loading annotations into memory...
Done (t=14.18s)
creating index...
index created!
Loaded train 118287 samples


In [9]:
img = next(iter(train_loader))

In [10]:
print(img.keys())
print(img['input'].size())

dict_keys(['input', 'ss', 'hm', 'reg_mask', 'ind', 'wh', 'reg'])
torch.Size([2, 3, 512, 512])


In [11]:
model.to('cuda')
torch.cuda.empty_cache()
outputs = model(img['input'].to('cuda'))

In [12]:
output = outputs[0]
print(output.keys())

dict_keys(['fc_ss', 'ss', 'hm', 'wh', 'reg'])


In [18]:
sum((output['ss'].softmax(dim=1))[0,:,0,0])

tensor(1.0000, device='cuda:0', grad_fn=<AddBackward0>)

In [13]:
from models.losses import *
from models.utils import _transpose_and_gather_feat

In [14]:
batch=img
mask = batch['reg_mask']
ind = batch['ind']
target_wh = batch['wh']
target_reg = batch['reg']
target_ss = batch['ss']

In [15]:
target_wh.size()

torch.Size([2, 128, 2])

In [16]:
target_ss.size()

torch.Size([2, 128])

In [17]:
print(output['wh'].size())
pred =_transpose_and_gather_feat(output['wh'].to('cuda'),ind.to('cuda'))
print(pred.size())
print(mask.size())
mask2 = mask.unsqueeze(2).expand_as(pred).float()
print(mask2.size())

torch.Size([2, 2, 128, 128])
torch.Size([2, 128, 2])
torch.Size([2, 128])
torch.Size([2, 128, 2])


In [18]:
print(output['ss'].size())
pred_ss =_transpose_and_gather_feat(output['ss'].to('cuda'),ind.to('cuda'))
print(pred_ss.size())
print(mask.size())
mask3 = mask.unsqueeze(2).expand_as(pred_ss).float()
print(mask3.size())

torch.Size([2, 80, 128, 128])
torch.Size([2, 128, 80])
torch.Size([2, 128])
torch.Size([2, 128, 80])


In [19]:
target_ss

tensor([[ 0, 33,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0],
        [62, 15,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          

In [20]:
target_ss[mask.bool()]

tensor([ 0, 33,  0, 62, 15])

In [21]:
pred_ss[mask.bool()]

tensor([[0.0525, 0.0044, 0.0217, 0.0047, 0.0152, 0.0132, 0.0239, 0.0258, 0.0214,
         0.0101, 0.0091, 0.0137, 0.0061, 0.0016, 0.0100, 0.0029, 0.0089, 0.0025,
         0.0042, 0.0439, 0.0061, 0.0022, 0.0006, 0.0046, 0.0367, 0.0077, 0.0040,
         0.0028, 0.0048, 0.0833, 0.0083, 0.0033, 0.0167, 0.0018, 0.0058, 0.0013,
         0.0044, 0.0137, 0.0105, 0.0066, 0.0223, 0.0039, 0.0031, 0.0546, 0.0099,
         0.0050, 0.0125, 0.0248, 0.0087, 0.0038, 0.0062, 0.0022, 0.0043, 0.0016,
         0.0053, 0.0036, 0.0023, 0.0057, 0.0022, 0.0065, 0.0046, 0.0010, 0.0058,
         0.0238, 0.0071, 0.0264, 0.0044, 0.0128, 0.0045, 0.0094, 0.0053, 0.0024,
         0.0080, 0.0286, 0.0148, 0.0022, 0.0211, 0.0033, 0.0959, 0.0089],
        [0.0027, 0.0137, 0.0037, 0.0198, 0.0051, 0.0225, 0.0245, 0.0102, 0.0236,
         0.0098, 0.0430, 0.0623, 0.0180, 0.0021, 0.0057, 0.0009, 0.0037, 0.0057,
         0.0121, 0.0216, 0.0159, 0.0026, 0.0038, 0.0005, 0.0319, 0.0093, 0.0073,
         0.0020, 0.0497, 0.0076, 0.

In [22]:
import torch.nn as nn

In [23]:
nn.CrossEntropyLoss()(pred_ss[mask.bool()].to('cuda'),target_ss[mask.bool()].to('cuda'))

tensor(4.3635, device='cuda:0', grad_fn=<NllLossBackward>)

In [24]:
F.cross_entropy(pred_ss[mask.bool()].to('cuda'),target_ss[mask.bool()].to('cuda'))

tensor(4.3635, device='cuda:0', grad_fn=<NllLossBackward>)

In [25]:
output['hm'].size()

torch.Size([2, 1, 128, 128])

In [26]:
batch['hm'].size()

torch.Size([2, 128, 128])

In [27]:
gt = batch['hm']
pred = torch.nn.Sigmoid()(output['hm'].squeeze(1))
print(gt.size())
print(pred.size())

torch.Size([2, 128, 128])
torch.Size([2, 128, 128])


In [28]:
pred.squeeze(1).size()

torch.Size([2, 128, 128])

In [33]:
def _slow_neg_loss(pred, gt):
  '''focal loss from CornerNet'''
  pos_inds = gt.eq(1)
  neg_inds = gt.lt(1)

  neg_weights = torch.pow(1 - gt[neg_inds], 4)

  loss = 0
  pos_pred = pred[pos_inds]
  neg_pred = pred[neg_inds]

  pos_loss = torch.log(pos_pred) * torch.pow(1 - pos_pred, 2)
  neg_loss = torch.log(1 - neg_pred) * torch.pow(neg_pred, 2) * neg_weights

  num_pos  = pos_inds.float().sum()
  pos_loss = pos_loss.sum()
  neg_loss = neg_loss.sum()

  if pos_pred.nelement() == 0:
    loss = loss - neg_loss
  else:
    loss = loss - (pos_loss + neg_loss) / num_pos
  return loss

In [46]:
_slow_neg_loss(pred.to('cuda'),gt.to('cuda'))

tensor(9.2038, device='cuda:0', grad_fn=<RsubBackward1>)

# Respecto a la deteccion

In [28]:
pred = torch.nn.Sigmoid()(output['hm'].squeeze(1))
batch=img
mask = batch['reg_mask']
ind = batch['ind']
target_wh = batch['wh']
target_reg = batch['reg']
target_ss = batch['ss']
target_hm = batch['hm']

In [29]:
from models.fewdecode import fewdet_decode
from models.fewdecode import _nms

In [30]:
batch, cat, height, width = output['ss'].size() 
print(output['ss'].size())

torch.Size([2, 80, 128, 128])


In [31]:
target_nms=_nms(target_hm)
print(target_nms.size())

torch.Size([2, 128, 128])


In [38]:
target_2d=target_nms.view(batch, -1)
print(target_2d.size())
mask_hm, topk_inds = torch.topk(target_2d.view(batch, -1), 100)
print(mask_hm.size(), topk_inds.size())

torch.Size([2, 16384])
torch.Size([2, 100]) torch.Size([2, 100])


In [39]:
topk_inds = topk_inds % (128*128)

In [44]:
topk_ys   = (topk_inds / width).int().float()
topk_xs   = (topk_inds % width).int().float()

In [47]:
pred = _transpose_and_gather_feat(output['ss'], topk_inds.to('cuda')) 

In [56]:
topk_score, topk_clses = torch.max(pred,dim=-1) 
topk_score, topk_clses = topk_score*mask_hm.to('cuda'),topk_clses*mask_hm.to('cuda')

In [58]:
topk_clses

tensor([[ 5., 78., 37.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.],
        [10.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.

In [57]:
topk_score

tensor([[0.0972, 0.0959, 0.0943, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000],
        [0.0988, 0.0733, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0